In [106]:
import re
import os
import pandas as pd
import emoji
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords, wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
import contractions

tqdm.pandas()

In [107]:
data = pd.read_csv("Data/GamesData.csv", usecols=["content", "score", "year", "game_name"])
data["length"] = data["content"].astype(str).str.len()
data = data[data["length"] <= 500]
data = data[data["score"] != 0]
data = data.dropna()
data = data.reset_index(drop=True)
# data = (
#     data.groupby("game_name", group_keys=False)
#         .apply(lambda x: x.sample(n=min(600, len(x)), random_state=42))
#         .reset_index(drop=True)
# )
data

,game_name,content,score,year,length
0,Garena Free Fire,Gunslinger meets battle royale! The developers...,5,2025,500
1,Garena Free Fire,Free Fire is a really fun and addictive battle...,4,2025,434
2,Garena Free Fire,I've been aware of this game for quite a while...,5,2025,496
3,Garena Free Fire,The game has the potential to be great. Howeve...,3,2019,405
4,Garena Free Fire,I personally find this game to be very enterta...,3,2019,499
...,...,...,...,...,...
2029346,Homescapes,Very boring. A lot of talk not enough action,1,2019,44
2029347,Homescapes,I love this house just wish the levels were ea...,4,2022,75
2029348,Homescapes,love this game only some of the levels seem im...,5,2019,63
2029349,Homescapes,Good game but requires in game payments which ...,3,2020,54


In [108]:
emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # emoticons 😀😅😂🤣😇🙃🙂
    "\U0001F300-\U0001F5FF"  # symbols & pictographs 🌍🌈⛄
    "\U0001F680-\U0001F6FF"  # transport & map symbols 🚀🚗🛳️
    "\U0001F1E0-\U0001F1FF"  # flags 🇮🇩🇺🇸
    "\U00002702-\U000027B0"  # dingbats ✂️✈️✔️
    "\U000024C2-\U0001F251"  # enclosed characters 🅰️🆗
    "]+",
    flags=re.UNICODE,
)

has_emoji = data["content"].astype(str).apply(lambda x: bool(emoji_pattern.search(x)))

print("Number of reviews containing emoji:", has_emoji.sum())
print("Percentage of reviews with emoji:", round(has_emoji.mean() * 100, 2), "%")

Number of reviews containing emoji: 197733
Percentage of reviews with emoji: 9.74 %


In [ ]:
def dedupe_mixed_emoji_bursts(text: str, case_insensitive=True, keep_cap=None):
    def repl(m):
        toks = re.findall(r':[^\s:]+:', m.group(0))
        seen = set()
        uniq = []
        for t in toks:
            key = t.lower() if case_insensitive else t
            if key not in seen:
                seen.add(key)
                uniq.append(t)
                if keep_cap is not None and len(uniq) >= keep_cap:
                    break
        return ' ' + ' '.join(uniq) + ' '

    return re.sub(r'(?:\s*:[^\s:]+:){2,}', repl, text)

In [ ]:
def expand_contractions(text):
    text = contractions.fix(text)
    text = re.sub(r"’", "'", text)
    return text

StopWords = set(stopwords.words("english"))
CustomStopWords = {"game", "play", "update", "get", "like", "good"}
StopWords = StopWords.union(CustomStopWords)

def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # fallback
    
Lemmatizer = WordNetLemmatizer()

In [111]:
def clean(review):
    #lower text
    review = review.lower()

    #translate emoji
    review = emoji.demojize(review, delimiters=(' :', ': '))
    review = dedupe_mixed_emoji_bursts(review, case_insensitive=True, keep_cap=10)
    review = review.replace("_", " ")
    review = re.sub(r":([a-z0-9 ]+):", r"\1", review)

    #expand contractions
    review = expand_contractions(review)

    #remove non-alphabet and non-numbers 
    review = re.sub(r'[^a-z0-9\s]+', '', review)    
    review = re.sub(r"\s+", " ", review).strip()
    if not review:
        return ""

    #tokenize words
    words = review.split()
    
    #lemmatize words
    tagged_words = pos_tag(words)
    lemmatized = [Lemmatizer.lemmatize(w, get_wordnet_pos(t)) for w, t in tagged_words]

    #remove stopwords
    cleaned = [w for w in lemmatized if w not in StopWords]

    return " ".join(cleaned)

In [ ]:
data['content'] = data['content'].progress_apply(clean)
data = data.dropna()
data.to_csv("Data/FullGamesDataClean.csv", index=False)
data

100%|██████████| 2029351/2029351 [34:44<00:00, 973.59it/s]  


,game_name,content,score,year,length
0,Garena Free Fire,gunslinger meet battle royale developer set ne...,5,2025,500
1,Garena Free Fire,free fire really fun addictive battle royale m...,4,2025,434
2,Garena Free Fire,aware quite refuse consider fan online shadow ...,5,2025,496
3,Garena Free Fire,potential great however unfair time instance l...,3,2019,405
4,Garena Free Fire,personally find entertaining however find need...,3,2019,499
...,...,...,...,...,...
2029346,Homescapes,bore lot talk enough action,1,2019,44
2029347,Homescapes,love house wish level easier l enjoy,4,2022,75
2029348,Homescapes,love level seem impossible clear,5,2019,63
2029349,Homescapes,require payment resent,3,2020,54
